1. fuzzer/seeds/test_seeds/zero-shot/221/264_263_243/seed_4.py

Executing seed file: /mnt/e/SoftwareCourses/DeepLearning/DlibFuzz/fuzzer/seeds/test_seeds/zero-shot/221/264_263_243/seed_4.py
PyTorch code failed: Calculated padded input size per channel: (1 x 3). Kernel size: (2 x 2). Kernel size can't be greater than actual input size
2024-09-08 18:21:31.212703: F ./tensorflow/core/util/tensor_format.h:427] Check failed: index >= 0 && index < num_total_dims Invalid index from the dimension: 3, 0, C
/tmp/tmpm_hzyz4w: line 3: 56558 Aborted                 python /mnt/e/SoftwareCourses/DeepLearning/DlibFuzz/oracle/driver.py
ERROR conda.cli.main_run:execute(125): `conda run python /mnt/e/SoftwareCourses/DeepLearning/DlibFuzz/oracle/driver.py` failed. (See above for error)

问题仍然出现的原因是由于在 TensorFlow 代码中引发了一个底层的崩溃错误，而这个错误并没有通过 Python 的异常捕获机制被处理。TensorFlow 有时在底层（C++部分）遇到问题时，会导致整个进程直接崩溃（Aborted），而不是抛出 Python 异常。

In [ ]:
import tensorflow
import jax
# Common variable values
input_tensor = [[1.0, 2.0, 3.0]]
kernel = [[0.5, 0.5], [0.5, 0.5]]

# PyTorch
import torch
import torch.nn.functional as F

input_pt = torch.tensor(input_tensor)
kernel_pt = torch.tensor(kernel)
output_pt = F.conv2d(input_pt.unsqueeze(0).unsqueeze(0), kernel_pt.unsqueeze(0).unsqueeze(0), stride=1)
print("PyTorch Output:", output_pt)

# TensorFlow
import tensorflow as tf

input_tf = tf.constant(input_tensor, dtype=tf.float32)
kernel_tf = tf.constant(kernel, dtype=tf.float32)
output_tf = tf.nn.conv2d(tf.expand_dims(input_tf, axis=0), tf.expand_dims(kernel_tf, axis=0), strides=[1, 1, 1, 1], padding='VALID')
print("TensorFlow Output:", output_tf.numpy())

# JAX
import jax.numpy as jnp
from jax import random

input_jax = jnp.array(input_tensor)
kernel_jax = jnp.array(kernel)
output_jax = jax.lax.conv_general_dilated(jnp.expand_dims(input_jax, axis=0).reshape(1, 1, 1, 3), 
                                           jnp.expand_dims(kernel_jax, axis=0).reshape(1, 1, 2, 2), 
                                           (1, 1), 
                                           'VALID')
print("JAX Output:", output_jax)